In [1]:
from SKLR.SKLR import SKLR
import torch
import numpy as np
import time

from scipy.spatial.distance import pdist, squareform
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics.pairwise import pairwise_kernels


#import importlib
#importlib.reload(some_module)




In [20]:
n=2048
d=2
X=np.random.rand(n,d)
Y=np.array([X[i,0]>0.2 for i in range(n)])
model_SKLR=SKLR(X,Y)
model_SKLR.fit()

/Users/mayuheng/Documents/GitHub/SketchLR/SKLR/SKLR.py:59: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/Users/mayuheng/Documents/GitHub/SketchLR/SKLR/SKLR.py:71: RuntimeWarning: divide by zero encountered in log
  return -np.log(cls.sigmoid(Y.dot( np.dot(KS,alpha))) ).sum()


KeyboardInterrupt: 

In [17]:
model_SKLR.accuracy

1.0

In [18]:
Y==model_SKLR.prediction

array([ True,  True,  True, ...,  True,  True,  True])